In [ ]:
!pip install jsonlines

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install pyldavis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 13.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyldavis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=062e84ed31ed02fd827dc28e12a1eb15d83248cd68d36cec1ba2b477547cf868
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=0541858063aacfc71768bc488ac7c17fcae2e8ef0d754ea14fec9b65435633b2
  Stored in directory: /root/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built pyldavis sklearn


In [ ]:
#spacy
import spacy
from spacy.pipeline import EntityRuler
from spacy.lang.en import English
from spacy.tokens import Doc

#gensim
import gensim
from gensim import corpora

#Visualization
from spacy import displacy
import pyLDAvis.gensim_models
from wordcloud import WordCloud
import plotly.express as px
import matplotlib.pyplot as plt

#Data loading/ Data manipulation
import pandas as pd
import numpy as np
import json
import jsonlines
from collections import Counter
#nltk
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download(['stopwords','wordnet', 'omw-1.4'])

#warning
import warnings 
#from PyPDF2 import PdfFileReader
from os.path import join

warnings.filterwarnings('ignore')
DATA_FILES = 'data/source_data/SkillDB'

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/tornado/httputil.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  class HTTPHeaders(collections.MutableMapping):
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!python -m spacy download fr_core_news_sm

2022-10-17 13:30:38.929278: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 16.3 MB 13.3 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')


In [ ]:
!python -m spacy download en_core_web_lg

2022-10-17 13:30:49.163557: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 587.7 MB 11 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
df = pd.read_csv(join(DATA_FILES, 'ESCO_FR', 'skills_fr.csv'))

In [ ]:
df.iloc[0,:]

conceptType                                KnowledgeSkillCompetence
conceptUri        http://data.europa.eu/esco/skill/0005c151-5b5a...
skillType                                          skill/competence
reuseLevel                                          sector-specific
preferredLabel                           gérer le personnel musical
altLabels                                                       NaN
hiddenLabels                                                    NaN
status                                                     released
modifiedDate                                   2016-12-20T17:43:43Z
scopeNote                                                       NaN
definition                                                      NaN
inScheme          http://data.europa.eu/esco/concept-scheme/skil...
description       Attribuer et gérer les tâches du personnel dan...
Name: 0, dtype: object

In [ ]:
Counter(list(df['reuseLevel']))

Counter({'sector-specific': 6654,
         'occupation-specific': 3043,
         'cross-sector': 3754,
         'transversal': 440,
         nan: 5})

In [ ]:
Counter(list(df['status']))

Counter({'released': 13896})

In [ ]:
# Counter(list(df['scopeNote']))

In [ ]:
Counter(list(df['skillType']))

Counter({'skill/competence': 10831, 'knowledge': 3059, nan: 6})

In [ ]:
Counter(list(df['conceptType']))

Counter({'KnowledgeSkillCompetence': 13896})

In [ ]:
df['preferredLabel']

0                               gérer le personnel musical
1               superviser des procédures correctionnelles
2        appliquer des pratiques de travail social anti...
3        contrôler la conformité à la réglementation su...
4                        recenser les services disponibles
                               ...                        
13891    restaurer les capacités professionnelles d’un ...
13892    installer l’éclairage dans des engins de trans...
13893            traitement automatique du langage naturel
13894               coordonner des travaux de construction
13895              placer des garde-corps et des bastaings
Name: preferredLabel, Length: 13896, dtype: object

In [ ]:
df = pd.read_csv(join(DATA_FILES, 'ESCO_FR', 'skills_fr.csv'))

In [ ]:
##df

In [ ]:
def write_json_lines(file_name,dict_data):
    json_string = json.dumps(dict_data)
    with open(file_name, 'a') as f:
        f.write(json_string+"\n")
        
def read_json_lines(file_name):
    lines = []
    with open(file_name) as file_in:
        for line in file_in:
            lines.append(json.loads(line))
    return lines


In [ ]:
def get_rules(skills_file, file_name):
  for ind, row in skills_file.iterrows():
      skill = row['name']
      rule_form = {}
      rule_form['label'] = 'SKILL'
      rule_form['pattern'] = [{'LOWER':word.lower()} for word in skill.split(' ')]
      write_json_lines(file_name,rule_form)

      
      # every rule {"label":"SKILL","pattern":[{"LOWER":"abstract"},{"LOWER":"data"},{"LOWER":"type"}]}

In [ ]:
file_name = join(DATA_FILES, 'skill_patterns.jsonl')

In [ ]:
get_rules(df, file_name)

KeyError: ignored

In [ ]:
read_json_lines(join(DATA_FILES, 'skill_patterns.jsonl'))

## 

In [ ]:
!pip -q install benepar

     |████████████████████████████████| 7.6 MB 20.3 MB/s 
     |████████████████████████████████| 5.3 MB 59.3 MB/s 
     |████████████████████████████████| 1.3 MB 67.8 MB/s 
     |████████████████████████████████| 163 kB 89.8 MB/s 


In [ ]:
import benepar, spacy

In [ ]:
!python -m spacy download en_core_web_md

2022-10-17 13:32:43.330675: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42.8 MB 1.2 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [ ]:
!python3 -q -m spacy download fr_core_news_md

2022-10-17 13:32:57.486502: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 45.8 MB 89.0 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_md')


In [ ]:
import nltk
benepar.download('benepar_fr2')
benepar.download('benepar_en3')

[nltk_data] Downloading package benepar_fr2 to /root/nltk_data...
[nltk_data]   Unzipping models/benepar_fr2.zip.
[nltk_data] Downloading package benepar_en3 to /root/nltk_data...
[nltk_data]   Unzipping models/benepar_en3.zip.


True

In [ ]:
def get_nlp_module(french = True):
    if french:
        lang_model = 'benepar_fr2'
        spacy_model = 'fr_core_news_md'
    else:
        lang_model = 'benepar_en3'
        spacy_model = 'en_core_web_md'
    nlp = spacy.load(spacy_model)
    if spacy.__version__.startswith('2'):
        nlp.add_pipe(benepar.BeneparComponent(lang_model))
    else:
        nlp.add_pipe("benepar", config={"model": lang_model})
    return nlp


In [ ]:
en_example = "The time for action is now. It's never too late to do something."
fr_example = 'assumer la responsabilité de préserver la sécurité de l’environnement d’un navire.'

In [ ]:
nlp = get_nlp_module(french = True)

In [ ]:
doc = nlp(fr_example)

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
sent = list(doc.sents)[0]

In [ ]:
print(sent._.parse_string)


(SENT (VPinf (VN (VINF assumer)) (NP (DET la) (NC responsabilité) (PP (P de) (VPinf (VN (VINF préserver)) (NP (DET la) (NC sécurité) (PP (P de) (NP (DET l’) (NC environnement) (PP (P d’) (NP (DET un) (NC navire)))))))))) (PONCT .))


In [ ]:
sent._.labels


('SENT',)

In [ ]:
list(sent._.children)[0]

assumer la responsabilité de préserver la sécurité de l’environnement d’un navire

In [ ]:
list(sent._.children)[1]

.

In [ ]:
list(sent._.children)[2]

IndexError: ignored

### Another Test

In [ ]:
import spacy   
from spacy.matcher import Matcher
from spacy.util import filter_spans                                                                                                                                                                                     

In [ ]:
nlp = spacy.load('en_core_web_sm')  # download model first

In [ ]:
sentence = 'The author was staring pensively as she wrote'

In [ ]:
pattern=[
  # {'POS': 'VERB', 'OP': '?'},
  # {'POS': 'VERB', 'POS': 'ADV'},
  {'POS': 'ADV', 'POS': 'VERB'}
#  {'POS': 'ADV', 'OP': '*'},
#  {'OP': '*'}, # additional wildcard - match any text in between
#  {'POS': 'VERB', 'OP': '+'}
 ]

In [ ]:
# instantiate a Matcher instance
matcher = Matcher(nlp.vocab)

In [ ]:
# Add pattern to matcher
matcher.add("verb-phrases", [pattern])


In [ ]:
doc = nlp(sentence)


In [ ]:
# call the matcher to find matches 
matches = matcher(doc)


In [ ]:
spans = [doc[start:end] for _, start, end in matches]

In [ ]:
spans

[staring, wrote]

In [ ]:
filter_spans(spans)

## Another Test

In [ ]:
nlp = spacy.load('fr_core_news_sm')  # download model first
# en_core_web_lg

In [ ]:
doc = nlp('aider des utilisateurs d’archives dans leurs recherches.')

In [ ]:
doc

aider des utilisateurs d’archives dans leurs recherches.

In [ ]:
list(doc.sents)

[aider des utilisateurs d’archives dans leurs recherches.]

In [ ]:
sent = list(doc.sents)[0]

In [ ]:
sent

aider des utilisateurs d’archives dans leurs recherches.

In [ ]:
def segment_skill(sent):
    all_sents = []
    all_sents_rep = []
    old_sent = ''
    old_phrase_sent = ''
    first_noun = False
    first_noun_ = True
    for idx, token in enumerate(sent):
        old_sent += ' '+ token.text
        if first_noun:
            old_phrase_sent += ' '+ token.text
        if (idx == 0) & (token.pos_ == 'VERB'):
            all_sents.append(old_sent.strip())
        elif (token.pos_ == 'NOUN') | (token.pos_ == 'ADJ'):
            first_noun  = True
            if len(old_sent.strip().split())>1:
                all_sents.append(old_sent.strip())
            if first_noun & first_noun_:
                old_phrase_sent += ' '+ token.text
                first_noun_ = False
            if len(old_phrase_sent.strip().split())>1:
                all_sents_rep.append(old_phrase_sent.strip())
    return list(set(all_sents + all_sents_rep))

In [ ]:
def get_sent_meta(sent):
    all_sents = []
    old_sent = ''
    for idx, token in enumerate(sent):
        old_sent += ' '+ token.text
        sent_meta = {}
        sent_meta['pos'] = token.pos_
        sent_meta['text'] = token.text
        sent_meta['lemma'] = token.lemma_
        sent_meta['i'] = token.i
        all_sents.append(sent_meta)
    return pd.DataFrame(all_sents)

def get_sub_phrases(sent):
    sent_df = get_sent_meta(sent)
    df_meta_ = sent_df[sent_df['pos'].isin(['NOUN','ADJ'])]
    df_meta_i = continued_sequence(list(df_meta_['i']))
    res = []
    for i in range(len(df_meta_i)):
        if i < len(df_meta_i)-1:
          phrase = ' '.join(list(sent_df[(sent_df['i']>=df_meta_i[i])&(sent_df['i']<=df_meta_i[i+1])]['text']))
          # print(phrase)
          res.append(phrase)
          phrase = ' '.join(list(sent_df[(sent_df['i']>=df_meta_i[i])&(sent_df['i']<=df_meta_i[i+1])]['lemma']))
          # print(phrase)
          res.append(phrase)
    return res

In [ ]:
def continued_sequence(tst):
    not_all = True
    prev_len = len(tst)
    new_len = len(tst)
    while not_all:
      for i in range(len(tst)):
          if i < len(tst)-1:
              if (i>1) & (tst[i+1]-tst[i]==1):
                  print(tst[i])
                  # tst[i] = tst[i+1]
                  del tst[i]
                  not_all = True
                  break
      new_len = len(tst)
      if new_len == prev_len:
          break
      else:
          prev_len = new_len
    return tst

In [ ]:
def segment_skill_v2(sent):
    all_sents = []
    old_sent = ''
    for idx, token in enumerate(sent):
        old_sent += ' '+ token.text
        # if token.pos_ == 'NOUN':
        return token

In [ ]:
df_meta = get_sent_meta(sent)

In [ ]:
df_meta

,pos,text,lemma,i
0,VERB,aider,aider,0
1,DET,des,un,1
2,NOUN,utilisateurs,utilisateur,2
3,ADV,d’,d’,3
4,ADJ,archives,archive,4
5,ADP,dans,dans,5
6,DET,leurs,leur,6
7,NOUN,recherches,recherche,7
8,PUNCT,.,.,8


In [ ]:
sent

aider des utilisateurs d’archives dans leurs recherches.

In [ ]:
get_sub_phrases(sent)

['utilisateurs d’ archives',
 'utilisateur d’ archive',
 'archives dans leurs recherches',
 'archive dans leur recherche']

In [ ]:
df_meta[df_meta['pos'].isin(['NOUN','ADJ'])]

,pos,text,lemma,i
2,NOUN,utilisateurs,utilisateur,2
4,ADJ,archives,archive,4
7,NOUN,recherches,recherche,7


In [ ]:
df[(df['pos']== 'NOUN') | (df['pos']== 'ADJ')]

,pos,text,lemma,i
2,NOUN,utilisateurs,utilisateur,2
4,ADJ,archives,archive,4
7,NOUN,recherches,recherche,7


In [ ]:
tst = [1, 2, 10, 15, 19, 20, 21, 22, 25, 28, 30, 31]

In [ ]:
continued_sequence(tst)

19
20
21
30


[1, 2, 10, 15, 22, 25, 28, 31]

In [ ]:
tst

[1, 2, 10, 15, 22, 25, 28, 31]

In [ ]:
dir(sent)

In [ ]:
segment_skill(sent)

['aider des utilisateurs d’ archives',
 'utilisateurs d’ archives',
 'utilisateurs d’ archives dans leurs recherches',
 'aider des utilisateurs',
 'aider des utilisateurs d’ archives dans leurs recherches',
 'aider']

In [ ]:
xx = segment_skill_v2(sent)

In [ ]:
xx.idx

0

In [ ]:
xx.i

0

In [ ]:
xx.ancestors

[]

In [ ]:
dir(xx)

In [ ]:
for i in sent:
  # print(i)
  print(i.text)
  print(i.pos_)
  if i.pos_ == 'VERB':
    print('..%')

In [ ]:
dir(i)